In [38]:
import json
import networkx as nx
from pdfminer.high_level import extract_text
import os
import re
import networkx as nx
import textdistance
import seaborn as sns


In [27]:
texts = []
for i in os.listdir("data/extraterritorial_cases"):
    t = extract_text(os.path.join("data/extraterritorial_cases", i))
    texts.append(t)

In [28]:
def get_name(astring):
    try:
        res = re.search(r"CASE OF.+\n", astring)
        res = res.group(0)
        return res.strip("\n")
    except:
        return "Unkown"

def get_appno(astring):
    try:
        res = re.search(r"\d{5}\/\d{2}|\d{4}\/\d{2}", astring)
        return res.group(0)
    except:
        return "Unkown"

def get_year(astring):
    try:
        res = re.search(r"\d{4}\n", astring)
        return res.group(0).strip("\n")
    except:
        return "Unkown"


In [29]:
storage = []
for i in texts:
    results = dict()
    results["name"] = get_name(i)
    results["appno"] = get_appno(i)
    results["year"] = get_year(i)
    results["text"] = i
    storage.append(results)


In [30]:
g = nx.Graph()

In [31]:
for i in storage:
    g.add_node(i["appno"], name = i["name"], year = i["year"], text = i["text"])

In [32]:
edge_storage = []
for i in storage:
    for j in storage:
        if i["appno"] == j["appno"]:
            pass
        else:
            pattern = i['appno']
            counts = re.findall(pattern, j["text"], flags=False)
            ncounts = len(counts)
            distance = textdistance.jaccard(i['text'], j['text'])
            g.add_edge(i["appno"], j["appno"], weight = ncounts, distance = distance)


In [37]:
dump = nx.node_link_data(g)
with open("data/extraterritorial_cases_3.json", "w") as outfile:
    json.dump(dump, outfile)


In [39]:
nx.degree(g, weight="weight")

DegreeView({'3394/03': 0, '15318/89': 0, '5809/08': 5, '51357/07': 6, '13216/05': 19, '25358/12': 0, '43611/11': 3, '1398/03': 3, '43370/04': 9, '27765/09': 4, '53600/20': 3, '36925/07': 7, '24384/19': 6, '3455/05': 0, '22854/20': 1, '48787/99': 13, '47708/08': 5, '38263/08': 5, '4871/16': 1, '35763/97': 14, '39630/09': 4, '11138/10': 3, '40167/06': 5, '8675/15': 2})

In [40]:
nx.degree(g, weight="distance")


DegreeView({'3394/03': 0.08060753309608937, '15318/89': 9.41452278408284, '5809/08': 11.751728113444699, '51357/07': 13.035417306277658, '13216/05': 11.60364588135044, '25358/12': 13.337879909089205, '43611/11': 13.340195005150083, '1398/03': 10.631047402990058, '43370/04': 12.980155244855519, '27765/09': 13.903663250498226, '53600/20': 8.264409965736165, '36925/07': 13.92010472636949, '24384/19': 13.526822596234132, '3455/05': 13.930139852526828, '22854/20': 8.073730750066408, '48787/99': 8.589468538175481, '47708/08': 13.942191351978643, '38263/08': 7.905115880346362, '4871/16': 13.985805403073384, '35763/97': 7.471502983150614, '39630/09': 13.764516436749048, '11138/10': 12.85256556934492, '40167/06': 12.26285858258719, '8675/15': 13.317810922581645})